# Sorghum growth

In [1]:
from __future__ import annotations

import os
import sys
import time as t
from datetime import date
from multiprocessing import Pool

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

sys.path.append("../data") 
from archi_dict import archi_sorghum_angles as archi
from oawidgets.plantgl import *  # noqa: F403

from openalea.archicrop.display import build_scene, display_scene  # noqa: F401
from openalea.archicrop.simulation import (
    plot_constained_vs_pot,
    plot_constrainted_vs_realized,
    plot_faPAR,
    plot_PAR,
    run_simulations,
    write_netcdf,
)
from openalea.plantgl.all import Color3, Material, Scene, Viewer  # noqa: F401

%gui qt

In [2]:
stics_color = "orange"
archicrop_color = "green"

In [3]:
tec_file_xml='../data/02NT18SorgV2D1_tec.xml'
plt_file_xml='../data/sorgho_imp_M_v10_plt.xml'
stics_output_file='../data/mod_s02NT18SorgV2D1.sti'
weather_file = '../data/ntarla_corr.2018'
location = {  
'longitude': 3.87,
'latitude': 12.58,
'altitude': 800,
'timezone': 'Europe/Paris'}

In [4]:
seed = 18

In [ ]:

start_time = t.time()

daily_dynamics, params_sets, pot_la, pot_h, realized_la, realized_h, nrj_per_plant, mtgs, filters, sowing_density = run_simulations(
    archi_params=archi, 
    tec_file=tec_file_xml, 
    plant_file=plt_file_xml, 
    dynamics_file=stics_output_file, 
    weather_file=weather_file,
    location=location,
    n_samples=3,
    latin_hypercube=False,
    opt_filter_organ_duration=False,
    opt_filter_pot_growth=False,
    opt_filter_realized_growth=False,
    light_inter=False,
    direct=False,
    error_LA_pot=1,
    error_height_pot=1,
    error_LA_realized=1,
    error_height_realized=1,
    seed=seed)

end_time = t.time()

elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time:.4f} minutes for {len(realized_la)} simulations")  # noqa: T201


Elapsed time: 0.2058 minutes for 8 simulations


In [6]:
# Retrieve STICS growth and senescence dynamics
dates = [value["Date"] for value in daily_dynamics.values()]
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]
par_incident = [value["Incident PAR"] for value in daily_dynamics.values()]
par_stics = [value["Absorbed PAR"] for value in daily_dynamics.values()]

In [7]:
write_netcdf("sorghum_growth_tillers", daily_dynamics, params_sets, pot_la, pot_h, realized_la, realized_h, nrj_per_plant, mtgs, filters, sowing_density, seed)

In [8]:
Viewer.display(build_scene(mtgs[3][-30])[0])

In [9]:
mtgs[3][-40].properties()['potential_growth_rate']

{3: 0.020645972222222222,
 4: 0.05948976909245817,
 5: 0.020645972222222222,
 6: 0.1262990534462912,
 7: 0.02064597222222222,
 8: 0.26118706081794996,
 9: 0.020645972222222225,
 10: 0.51908394398122,
 11: 0.020645972222222232,
 12: 0.9781344007999893,
 13: 0.2858346908207814,
 14: 1.724150195486833,
 15: 0.25725122173870324,
 16: 2.8048388112310803,
 17: 0.23152609956483286,
 18: 4.154680156563806,
 19: 0.20837348960835,
 20: 5.52846778468597,
 21: 0.18753614064751456,
 22: 6.520037532197943,
 23: 0.16878252658276313,
 24: 6.723790267873296,
 25: 0.15190427392448677,
 26: 5.981878610866598,
 27: 0.13671384653203844,
 28: 4.529612319764924,
 29: 0.12304246187883426,
 30: 2.88021624607692,
 31: 0.11073821569095142,
 32: 1.5172963330254794,
 33: 0.09966439412185564,
 34: 0.6533375140892612}